In [1]:
from scrape_courses import get_courses_from_dgscene, get_readable_course_name
from scrape_events import get_all_sanctioned_events
from scrape_ratings import get_round_ratings_for_tournament
from logger import logger
import json
import itertools

In [ ]:
# TODO
# generate list of all courses
# write to json
courses = get_courses_from_dgscene()
course_names = {
    course: get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)

with open('data/course_events.json') as f:
    course_events: dict = json.load(f)

for i, course in enumerate(course_names):
    if course in course_events:
        logger.info(f'Skipping {course} (already scraped)...')
        continue

    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = get_all_sanctioned_events(course)
    logger.info(course_events[course])

    # periodically save ratings to file
    with open('data/course_events.json', 'w') as f:
        json.dump(course_events, f, indent=4)

with open('data/course_events.json', 'w') as f:
    json.dump(course_events, f, indent=4)

[15/Oct/2024 16:07:48] INFO - Skipping 22_Birdwalk (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping 4_Wing_Campground (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Alexander_Hamilton_Park (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Anderson_Park_AB (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Aperture_Park (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Aspen_Meadows (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Badlands_DGC (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Baileys_Crossing_Disc_Golf_Course (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Baker_Park_AB (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Barrett_Park (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Bear_Lake_Open (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping bear-paw-par-3-12330 (already scraped)...
[15/Oct/2024 16:07:48] INFO - Skipping Bonavista_Downs_Community (alrea

In [ ]:
try:
    logger.info('Fetching ratings...')
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    with open('data/course_ratings.json') as f:
        course_ratings: dict = json.load(f)

    for i, course in enumerate(course_events):
        if course in course_ratings:
            logger.info(f'Skipping {course} (already scraped)...')
            continue
        course_ratings[course] = {}
        events = course_events[course]

        for j, event in enumerate(events):
            course_ratings[course][event] = get_round_ratings_for_tournament(event)
            logger.info(
                f'Event {j+1}/{len(events)} - Course {i+1}/{len(course_events)}')

        # periodically save ratings to file
        with open('data/course_ratings.json', 'w') as f:
            json.dump(course_ratings, f, indent=4)

except BaseException as e:
    logger.info(f'Error fetching ratings: {e}')
except KeyboardInterrupt as e:
    logger.info(f'Error fetching ratings: {e}')

with open('data/course_ratings.json', 'w') as f:
    json.dump(course_ratings, f, indent=4)
logger.info('Ratings dumped to course_ratings.json.')
logger.info('')

In [ ]:
logger.info('Compiling data...')
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)
with open('data/course_events.json') as f:
    course_events: dict = json.load(f)
with open('data/course_ratings.json') as f:
    course_ratings: dict = json.load(f)

course_data = {}
for course in course_names:
    rounds = []
    for event_id, round_data in course_ratings.get(course, {}).items():
        round_data.put('event_id', event_id)
        round_data.put(course_events)
        rounds.extend(round_data)
    course_data[course] = {
        'name': course_names[course],
        'rounds': list(itertools.chain.from_iterable([round_data for event_id, round_data in course_ratings.get(course, {}).items()]))
    }

course_data = {
    course: {
        'name': course_names[course],
        'rounds': list(itertools.chain.from_iterable([round_data for event_id, round_data in course_ratings.get(course, {}).items()]))
    } for course in course_names
}
with open('data/course_data.json', 'w') as f:
    json.dump(course_data, f, indent=4)
logger.info('Data compiled and dumped to course_data.json.')
logger.info('')
